## ETL:
#### 1 - Extraccion

1. Carga de datos del archivo "movies_dataset": Utilizando la biblioteca pandas, se cargó el archivo movies_dataset.csv y se realizó una inspección inicial para evaluar la estructura de datos, columnas y tipos de datos presentes. Se visualizaron las primeras filas para una revisión preliminar.

In [ ]:
import pandas as pd

# Cargamos el archivo
movies_dataset_df = pd.read_csv("movies_dataset.csv")

# Realizamos una inspección inicial
print(movies_dataset_df.info())  # Información general de las columnas y tipos de datos

In [ ]:
movies_dataset_df.head()  # Primeras filas del DataFrame

2. Carga de datos del archivo "credits": De igual manera, se cargó el archivo credits.csv y se realizó la misma inspección inicial de estructura y tipos de datos, con el objetivo de comprender los datos para su posterior transformación.

In [ ]:
# Cargamos el archivo
credits_df = pd.read_csv("credits.csv")

# Realizamos una inspección inicial
print(credits_df.info())  # Información general de las columnas y tipos de datos

In [ ]:
credits_df.head()  # Primeras filas del DataFrame

#### 2- Transformacion

Transformación del dataset "movies_dataset":

Eliminación de columnas irrelevantes: Se suprimieron columnas no requeridas (video, imdb_id, adult, original_title, poster_path, homepage) para optimizar el dataset según las necesidades del cliente.

In [ ]:
# Eliminamos columnas innecesarias
movies_dataset_df = movies_dataset_df.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'])

# Confirmamos la eliminación de las columnas
print(movies_dataset_df.columns)

Revisión y tratamiento de valores faltantes: Se identificaron y contaron los valores nulos por columna, y en el caso de las columnas revenue y budget, se rellenaron con ceros. Estos campos fueron convertidos a tipo numérico para asegurar un correcto procesamiento y análisis.

In [ ]:
# Revisamos los valores faltantes
missing_values = movies_dataset_df.isnull().sum().sort_values(ascending=False) #recuento de valores nulos en orden descendente
print("Valores faltantes por columna:")
print(missing_values)

Creación de columna de retorno (return): Para esta columna, se calculó la relación entre revenue y budget, asignando cero cuando budget tenía un valor nulo o era igual a cero.

In [ ]:
# Rellenamos valores nulos en las columnas 'revenue' y 'budget' con 0
movies_dataset_df['revenue'] = movies_dataset_df['revenue'].fillna(0)
movies_dataset_df['budget'] = movies_dataset_df['budget'].fillna(0)

# Confirmamos de que ya no hay valores nulos en 'revenue' y 'budget'
print("Valores nulos en 'revenue':", movies_dataset_df['revenue'].isnull().sum())
print("Valores nulos en 'budget':", movies_dataset_df['budget'].isnull().sum())

In [ ]:
# Convertimos 'budget' y 'revenue' a numérico, reemplazando valores no convertibles por 0
movies_dataset_df['budget'] = pd.to_numeric(movies_dataset_df['budget'], errors='coerce')
movies_dataset_df['revenue'] = pd.to_numeric(movies_dataset_df['revenue'], errors='coerce')

# Creamos la columna 'return' dividiendo 'revenue' entre 'budget'
movies_dataset_df['return'] = movies_dataset_df.apply(lambda row: row['revenue'] / row['budget'] if row['budget'] > 0 else 0, axis=1)

# Confirmamos la creación de la nueva columna
print(movies_dataset_df[['revenue', 'budget', 'return']].head())

Transformación de la columna release_date: Se eliminaron las filas con valores nulos en release_date, se convirtió al formato de fecha estándar (AAAA-MM-DD) y se generó la columna release_year extrayendo el año de la fecha de lanzamiento.

In [ ]:
movies_dataset_df["release_date"].isna().value_counts()

In [ ]:
# Eliminamos filas con valores nulos en la columna 'release_date'
movies_dataset_df = movies_dataset_df.dropna(subset=['release_date'])
# Confirmamos la eliminación
print("Valores nulos en 'release_date' después de la eliminación:", movies_dataset_df['release_date'].isnull().sum())

In [ ]:
# Convertimos 'release_date' al formato AAAA-mm-dd
movies_dataset_df['release_date'] = pd.to_datetime(movies_dataset_df['release_date'], errors='coerce').dt.strftime('%Y-%m-%d')

# Creamos la columna 'release_year' extrayendo solo el año
movies_dataset_df['release_year'] = pd.to_datetime(movies_dataset_df['release_date'], errors='coerce').dt.year

# Confirmamos los cambios
print(movies_dataset_df[['release_date', 'release_year']].head())



Desanidado de columnas anidadas: Con ayuda de la librería ast, se desanidaron los valores en las columnas belongs_to_collection, production_companies, genres, production_countries y spoken_languages, extrayendo solo la información relevante como nombres de colecciones, compañías de producción, géneros, países de producción y lenguajes hablados.

In [ ]:
movies_dataset_df["belongs_to_collection"].head()

In [ ]:
#Importamos la libreria ast (arboles de sintaxis abrstracta)
import ast 

# Convertimos a diccionario y extraemos el nombre 'name' de la colección si existe
movies_dataset_df['belongs_to_collection'] = movies_dataset_df['belongs_to_collection'].apply(
    lambda x: ast.literal_eval(x).get('name') if pd.notnull(x) and isinstance(x, str) and x.startswith('{') else None)

In [ ]:
movies_dataset_df["belongs_to_collection"].head()

In [ ]:
movies_dataset_df["production_companies"].head()

In [ ]:
# Extraemos los nombres de las compañías de producción
movies_dataset_df['production_companies'] = movies_dataset_df['production_companies'].apply(
    lambda x: [i['name'] for i in ast.literal_eval(x)] if pd.notnull(x) and isinstance(x, str) and x.startswith('[') else None
)


In [ ]:
movies_dataset_df["production_companies"].head()

In [ ]:
movies_dataset_df["genres"].head()

In [ ]:
# Extramos los nombres de la columna géneros
movies_dataset_df['genres'] = movies_dataset_df['genres'].apply(
    lambda x: [i['name'] for i in ast.literal_eval(x)] if pd.notnull(x) and isinstance(x, str) else None)

In [ ]:
movies_dataset_df["genres"].head()

In [ ]:
movies_dataset_df["production_countries"].head()

In [ ]:
# Extraemos los nombres de las compañías de producción
movies_dataset_df['production_countries'] = movies_dataset_df['production_countries'].apply(
    lambda x: [i['name'] for i in ast.literal_eval(x)] if pd.notnull(x) and isinstance(x, str) and x.startswith('[') else None)

In [ ]:
movies_dataset_df["production_countries"].head(10)

In [ ]:
movies_dataset_df["spoken_languages"].head()

In [ ]:
movies_dataset_df['spoken_languages'] = movies_dataset_df['spoken_languages'].apply(
    lambda x: [i['name'] for i in ast.literal_eval(x)] if pd.notnull(x) and isinstance(x, str) and x.startswith('[') else None)

In [ ]:
movies_dataset_df["spoken_languages"].head()

Revisamos nuevamente el Data Frame

In [ ]:
movies_dataset_df

Filtrado de idiomas: Para reducir el tamaño del archivo y centrarse en los idiomas principales, se seleccionaron las filas que contenían solo los cinco idiomas más frecuentes: English, Spanish, Français, Italiano y Portuguese.

In [ ]:
top_5_idiomas = movies_dataset_df["spoken_languages"].value_counts()
top_5_idiomas


In [ ]:
# Define los idiomas deseados
idiomas_deseados = ['English', 'Spanish', 'Français',"Italiano", 'Portuguese']

# Filtra el dataframe para mantener solo las filas que contienen los idiomas deseados en 'spoken_languages'
movies_dataset_df = movies_dataset_df[
    movies_dataset_df['spoken_languages'].apply(lambda idiomas: any(idioma in idiomas_deseados for idioma in idiomas) if idiomas else False)
]

# Revisa el resultado
movies_dataset_df


Nuevamente revisamos el dataframe y vemos que sigue habendo columnas que no serian necesarias para cumplir con los requerimientos por lo que procedemos a eliminarlas

In [ ]:
#Revisamos las columnas esistentes en el dataframe
print(movies_dataset_df.columns)

Eliminación adicional de columnas innecesarias: Tras una última revisión, se eliminaron las columnas overview, tagline, status, spoken_languages, original_language y popularity para ajustar el dataset a los requerimientos.

In [ ]:
movies_dataset_df = movies_dataset_df.drop(columns=['overview', 'tagline', 'status', 'spoken_languages', 'original_language', 'popularity'])  

#### 2- Transformación del dataset "credits":

In [ ]:
credits_df

In [ ]:
credits_df["cast"].head()

Extracción de información específica de cast y crew: Mediante una función personalizada, se extrajeron de la columna cast únicamente los datos de character y name y, de la columna crew, se extrajeron solo los nombres de los directores, manteniendo así los detalles clave requeridos.

In [ ]:
#Importamos la libreria ast (arboles de sintaxis abstracta)
import ast 
# Creamos una funcion para extraer solo 'character' y 'name' de la columna 'cast'
def extraer_cast_info(cast_data):
    try:
        cast_list = ast.literal_eval(cast_data) # Intenta convertir de cadena a lista de diccionarios
        return [{'character': person['character'], 'name': person['name']} for person in cast_list] #De poder hacerlo el paso anterior, extrae solo 'character' y 'name'
    except (ValueError, TypeError):
        return None # De no poder convertir a diccionario por error de valor o tipo devuelve None

# Aplicamos la función al DF para crear una nueva columna 'cast_info'
credits_df['cast_info'] = credits_df['cast'].apply(extraer_cast_info)

In [ ]:
credits_df["cast_info"].head()

In [ ]:
# Creamos una unción para extraer sólo el nombre de los directores de la columna 'crew'
def extraer_director_info(crew_data):
    try:
        crew_list = ast.literal_eval(crew_data)# Intenta convertir de cadena a lista de diccionarios
        return [person['name'] for person in crew_list if person['job'] == 'Director'] #De poder hacerlo, filtra solo los directores y extrae sus nombres
    except (ValueError, TypeError): # De no poder convertir a diccionario por error de valor o tipo devuelve None
        return None

# Aplicamos la función al DF para crear una nueva columna 'director_info'
credits_df['director_info'] = credits_df['crew'].apply(extraer_director_info)

In [ ]:
credits_df['director_info'].head()

Limpieza de columnas originales: Una vez creada la información necesaria en nuevas columnas (cast_info y director_info), se eliminaron las columnas cast y crew para simplificar el dataset.

In [ ]:
credits_df.drop(columns=["cast","crew"],inplace=True)

In [ ]:
credits_df

#### 3- Carga

Concatenación y guardado final: Los DataFrames transformados (movies_dataset_df y credits_df) fueron concatenados en un solo DataFrame, el cual fue guardado en un nuevo archivo llamado DF_final.csv, que contiene los datos finales listos para análisis y consulta.

In [ ]:
Df_limpio = pd.concat([movies_dataset_df,credits_df],axis=1)

In [ ]:
Df_limpio.to_csv("DF_final.csv",index=False)

### Sistema de Recomendación Basado en Similitud de Coseno

Para desarrollar un sistema de recomendación de películas, utilizaremos la similitud de coseno, que nos permite medir la similitud entre vectores de características. A continuación, detallamos cada paso del proceso.

Paso 1: Importación de Librerías y Carga de Datos

Primero, importamos las librerías necesarias y cargamos el archivo preprocesado, DF_final.csv, que contiene las columnas y valores listos para construir el sistema de recomendación.

In [ ]:
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Cargar el dataset final
df_modelo = pd.read_csv("DF_final.csv")
print("Dimensiones del dataset:", df_modelo.shape)

Paso 2: Selección y Limpieza de Parámetros Clave

En este paso, seleccionamos las columnas de interés para el sistema de recomendación. Las características numéricas incluyen release_year, vote_average, vote_count, y runtime. Las características categóricas abarcan genres, production_companies, y production_countries.

In [ ]:
# Convertir columnas en listas si aún no lo están
df_modelo['genres'] = df_modelo['genres'].apply(lambda x: eval(x) if isinstance(x, str) else [])
df_modelo['production_companies'] = df_modelo['production_companies'].apply(lambda x: eval(x) if isinstance(x, str) else [])
df_modelo['production_countries'] = df_modelo['production_countries'].apply(lambda x: eval(x) if isinstance(x, str) else [])


Para simplificar el modelo, filtramos las 50 compañías de producción y países más comunes, manteniendo solo estos en cada película.

In [ ]:
# Filtrar las 50 compañías y países más comunes
top_companies = df_modelo['production_companies'].explode().value_counts().head(50).index
top_countries = df_modelo['production_countries'].explode().value_counts().head(50).index

df_modelo['production_companies'] = df_modelo['production_companies'].apply(lambda x: [i for i in x if i in top_companies])
df_modelo['production_countries'] = df_modelo['production_countries'].apply(lambda x: [i for i in x if i in top_countries])


Paso 3: Codificación de Características Categóricas


Utilizamos MultiLabelBinarizer para transformar las columnas de géneros, compañías de producción y países en variables binarias.

In [ ]:
# Codificación de géneros, compañías de producción y países de producción
mlb_genres = MultiLabelBinarizer()
genres_encoded = mlb_genres.fit_transform(df_modelo['genres'])

mlb_companies = MultiLabelBinarizer()
companies_encoded = mlb_companies.fit_transform(df_modelo['production_companies'])

mlb_countries = MultiLabelBinarizer()
countries_encoded = mlb_countries.fit_transform(df_modelo['production_countries'])


Paso 4: Normalización de Características Numéricas

Para las características numéricas, usamos MinMaxScaler para que todos los valores se encuentren en un rango entre 0 y 1, facilitando la comparación en la matriz de similitud.

In [ ]:
# Normalización de características numéricas
scaler = MinMaxScaler()
numerical_features = df_modelo[['release_year', 'vote_average', 'vote_count', 'runtime']].fillna(0)
numerical_scaled = scaler.fit_transform(numerical_features)


Paso 5: Combinación de Características

Fusionamos las características numéricas normalizadas y las variables binarias en un solo conjunto de características, listo para el cálculo de similitud de coseno.

In [ ]:
# Combinación de todas las características en un solo array de características
caracteristicas = np.hstack([numerical_scaled, genres_encoded, companies_encoded, countries_encoded])
print("Shape de las características combinadas:", caracteristicas.shape)


Paso 6: Creación de DataFrame de Características

Creamos un nuevo DataFrame, df_caracteristicas, que contiene todas las características necesarias para el sistema de recomendación y añadimos el índice original para referencia.

In [ ]:
# Crear DataFrame de características y añadir el índice original
df_caracteristicas = pd.DataFrame(caracteristicas)
df_caracteristicas['original_index'] = df_modelo.index
df_caracteristicas['title'] = df_modelo['title'].values  # Añadir columna título


Guardamos este DataFrame para futuras referencias.

In [ ]:
df_caracteristicas.to_csv("df_caracteristicas.csv", index=False)


Paso 7: Cálculo de Similitud de Coseno

Finalmente, calculamos la similitud de coseno entre las películas para recomendar aquellas que son más similares.

In [ ]:
# Cálculo de la similitud de coseno
matriz_caracteristicas = df_caracteristicas.drop(columns=['original_index', 'title']).values
similitud_de_coseno = cosine_similarity(matriz_caracteristicas)